In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [8]:
treino=pd.read_csv("poker-hand-training-true.data",header=None)
teste=pd.read_csv("poker-hand-testing.data",header=None)
coluna=['Naipe carta (1)','Valor carta (1)','Naipe carta (2)','Valor carta (2)','Naipe carta (3)','Valor carta (3)','Naipe carta (4)','Valor carta (4)','Naipe carta #5','Valor carta 5','Mão']
treino.columns=coluna
teste.columns=coluna

treino

,Naipe carta (1),Valor carta (1),Naipe carta (2),Valor carta (2),Naipe carta (3),Valor carta (3),Naipe carta (4),Valor carta (4),Naipe carta #5,Valor carta 5,Mão
0,1,10,1,11,1,13,1,12,1,1,9
1,2,11,2,13,2,10,2,12,2,1,9
2,3,12,3,11,3,13,3,10,3,1,9
3,4,10,4,11,4,1,4,13,4,12,9
4,4,1,4,13,4,12,4,11,4,10,9
...,...,...,...,...,...,...,...,...,...,...,...
25005,3,9,2,6,4,11,4,12,2,4,0
25006,4,1,4,10,3,13,3,4,1,10,1
25007,2,1,2,10,4,4,4,1,4,13,1
25008,2,12,4,3,1,10,1,12,4,9,1
